In [23]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [24]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [25]:
DF=pd.read_csv('BAN_list_Not_paid_greater_than_45_days.csv')

In [26]:
DF.head()

,BAN
0,602808606
1,605273507
2,604321916
3,605219027
4,601758133


In [8]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'pras-pr-223186.pras_pr_dataset.BAN_list_Not_paid_greater_than_45_days_01Mar2024'

bq_table_instance= bq_client.load_table_from_dataframe(DF, Table_BQ,job_config=config)

In [16]:
Sales_order_Query='''


with original
as 
(SELECT 
min(dly_ord_itm_actvy_ts) as first_add,
bus_prod_instnc_id
 FROM `cio-datahub-enterprise-pr-183a.ent_cust_ord_actvy.bq_dly_wln_ord_item_actvy` WHERE ord_act_typ_cd = 'Add' and prod_typ_cd in ('SMHM') and ord_act_stat_cd = 'Processed' and bus_prod_instnc_id is not null
 --and date(dly_ord_itm_actvy_ts) >= '2023-10-01' and date(dly_ord_itm_actvy_ts) < '2024-01-01'

 group by bus_prod_instnc_id) /* Fetch original order date for the product instance */



,BAN_list as (

select * from `pras-pr-223186.pras_pr_dataset.BAN_list_Not_paid_greater_than_45_days_01Mar2024`

)

,details as
(
SELECT 
dly_ord_itm_actvy_ts,
bus_billg_acct_num,
bus_prod_instnc_id,
prod_nm,
munic_nm,
prov_state_cd,
bi_chnl_tag_cd,
chnl_org_id,
prod_typ_cd,
src_sls_rep_cd,
src_typ_cd,
src_usr_chnl_txt,
txn_sub_typ_txt,
SLS_ACTVY_TXT
 FROM `cio-datahub-enterprise-pr-183a.ent_cust_ord_actvy.bq_dly_wln_ord_item_actvy` WHERE ord_act_typ_cd = 'Add' 
 --and prod_typ_cd in ('SMHM','HSIC','SING','TTV') 
 and prod_typ_cd in ('SMHM')
 and ord_act_stat_cd = 'Processed'
 --and date(dly_ord_itm_actvy_ts) >= '2023-10-01' and date(dly_ord_itm_actvy_ts) < '2024-01-01' 
 
 )
 
 
 
  /* Gather channel details */

,channel_refs as (


with
latest_update as (
SELECT
chnl_org_id,
max(chnl_org_key) as latest_key FROM `cio-datahub-enterprise-pr-183a.ent_sls_chnl.bq_channel_org_dim`
group by chnl_org_id

)

select distinct
t1.chnl_org_id,
chnl_org_txt from `cio-datahub-enterprise-pr-183a.ent_sls_chnl.bq_channel_org_dim` t1 inner join latest_update on latest_key = chnl_org_key

)

,Sales_Order_DF as (select
date(details.dly_ord_itm_actvy_ts) as Order_date,
details.bus_billg_acct_num as BAN,
details.bus_prod_instnc_id as Prod_Instnc_ID,
details.prod_nm as Prod_nm,
details.munic_nm,
details.prov_state_cd,
details.bi_chnl_tag_cd as Sales_Channel_tag,
details.chnl_org_id,
channel_refs.chnl_org_txt,
details.prod_typ_cd as Prod_Type,
details.src_sls_rep_cd as Sales_Agent_ID,
details.src_typ_cd,
details.src_usr_chnl_txt,
details.txn_sub_typ_txt,
details.SLS_ACTVY_TXT
from details inner join original on original.bus_prod_instnc_id = details.bus_prod_instnc_id and original.first_add = details.dly_ord_itm_actvy_ts  /* Inner join them to prevent duplicates */
 left join channel_refs on details.chnl_org_id = channel_refs.chnl_org_id /* left joining to get channel dealer names */
 where bi_chnl_tag_cd <> 'UNKNOWN' 
 order by details.dly_ord_itm_actvy_ts
)


select a.*,b.*
from BAN_list a 
left join Sales_Order_DF b
on a.BAN=b.BAN

'''

In [17]:
DF_sales_Order=extract_bq_data(bq_client, sql=Sales_order_Query)

In [18]:
DF_sales_Order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68358 entries, 0 to 68357
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BAN                68358 non-null  Int64 
 1   Order_date         54873 non-null  dbdate
 2   BAN_1              54873 non-null  Int64 
 3   Prod_Instnc_ID     54873 non-null  object
 4   Prod_nm            54873 non-null  object
 5   munic_nm           54873 non-null  object
 6   prov_state_cd      54873 non-null  object
 7   Sales_Channel_tag  54873 non-null  object
 8   chnl_org_id        54873 non-null  Int64 
 9   chnl_org_txt       54873 non-null  object
 10  Prod_Type          54873 non-null  object
 11  Sales_Agent_ID     54873 non-null  object
 12  src_typ_cd         54873 non-null  object
 13  src_usr_chnl_txt   54873 non-null  object
 14  txn_sub_typ_txt    51953 non-null  object
 15  SLS_ACTVY_TXT      51953 non-null  object
dtypes: Int64(3), dbdate(1), object(12)
memor

In [19]:
DF_sales_Order.head()

,BAN,Order_date,BAN_1,Prod_Instnc_ID,Prod_nm,munic_nm,prov_state_cd,Sales_Channel_tag,chnl_org_id,chnl_org_txt,Prod_Type,Sales_Agent_ID,src_typ_cd,src_usr_chnl_txt,txn_sub_typ_txt,SLS_ACTVY_TXT
0,601072960,2021-02-12,601072960,1113765242,Smart Automation Plus (Automation Hub),RED DEER,AB,CSD,-1000000000,UNKNOWN,SMHM,t875030,FIFA,FFH_CAR_FBR_BC,New Contract,Techs - CSD
1,605269194,2022-07-05,605269194,1128252300,Secure Plus Video,CALGARY,AB,CSD,-1000000000,UNKNOWN,SMHM,t899226,FIFA,INTERNAL_TECH,New Contract,Techs - CSD
2,603666766,2023-05-19,603666766,1141056231,Smart Automation Plus,SHERWOOD PARK,AB,MU8,-1000000000,UNKNOWN,SMHM,t873560,FIFA,Frontline,New Contract,N/AVAIL
3,603822822,2020-03-12,603822822,1109792626,Smart Automation Plus Video,EDMONTON,AB,CSD,-1000000000,UNKNOWN,SMHM,t931709,FIFA,INTERNAL_TECH,New Contract,Techs - CSD
4,604679524,2021-05-26,604679524,1115286744,Secure Plus Video,Brampton,ON,DEALER_ADT_OB,11918486,TELUS Internal Field Sales,SMHM,UKN,FIFA,Mississauga,New Contract,Channels - Corporate Store


In [21]:
DF_sales_Order.to_csv('Not_Paid_Mar2024.csv',index=False)